# Dependências


In [ ]:
!pip -q install langchain "openai<1.0.0" tiktoken "pinecone-client[grpc]" apache_beam mwparserfromhell cohere python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) .

# Carregar *API keys*

In [ ]:
from google.colab import files
from dotenv import load_dotenv
files_uploaded = files.upload()
load_dotenv()

Saving .env to .env


True

# Imports

In [ ]:
# Gerais
import os
import datetime
from uuid import uuid4
import tiktoken
import pinecone

# LangChain
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.schema import BaseOutputParser

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Definição de funções

In [ ]:
# Função de comprimento (len) baseado em um tokenizer
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

In [ ]:
# Função de inserção de dados em um index do Pinecone
def insert_index(index, texts, metadatas, embeddings_model):
    # Obtem ids unicos
    ids = [str(uuid4()) for _ in range(len(texts))]
    # Realiza embedding dos textos
    embeds = embeddings_model.embed_documents(texts)
    # Insere no índice
    index.upsert(vectors=zip(ids, embeds, metadatas))

In [ ]:
# Função que itera sobre os chunks para inseri-los no Pinecone
def insert_pinecone(index, chunks, embedding_model):
  batch_limit = 100
  texts = []
  metadatas = []

  # Chunks obtidos na módulo de divisao
  for chunk in chunks:
      # Obter texto e metadados do chunk
      chunk_text = chunk.page_content
      chunk_metadatas = chunk.metadata
      chunk_metadatas["year"] = datetime.datetime.now().year
      chunk_metadatas["text"] = chunk_text

      # Adiciona a lista de textos e metadados
      texts.append(chunk_text)
      metadatas.append(chunk_metadatas)

      # Se ultrapassou o tamanho limite do lote (batch),
      # insere no índice
      if len(texts) >= batch_limit:
          insert_index(index, texts, metadatas, embedding_model)
          # Esvazia listas
          texts = []
          metadatas = []

  # Se ainda restam dados a serem inseridos
  if len(texts) > 0:
      insert_index(index, texts, metadatas, embedding_model)

In [ ]:
def get_sources(vectorstore, query, k=3):
    return vectorstore.similarity_search(query, k=k)

# Alimentar banco de dados


In [ ]:
# Carregar documentos

loader = WebBaseLoader([
    "https://jornal.usp.br/ciencias/ciencias-isolamento-e-coesao-dos-grupos-de-direita-facilitaram-propagacao-coordenada-nas-eleicoes/",
    "http://www.saocarlos.usp.br/atencao-a-saude-mental-e-inclusao-na-universidade/",
    "https://cemeai.icmc.usp.br/projeto-tematico-une-ciencia-de-dados-e-sociologia-no-mapeamento-da-criminalidade/",
    "https://cemeai.icmc.usp.br/o-avanco-das-pesquisas-matematicas-com-foco-no-espectro-autista/"
])

documents = loader.load()

# Definição do tokenizer
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Definição do text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

# Divide o documento em chunks
chunks = text_splitter.split_documents(documents)

# Conecta ao Pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

# Verifica se é preciso criar o índice
index_name = "noticias-icmc"
if index_name not in pinecone.list_indexes():
    # Cria novo índice
    pinecone.create_index(
        name=index_name,
        metric="cosine", # Metrica de busca
        dimension=1536  # Dimensão do embedding. 1536 para text-embedding-ada-002
    )

# Carrega o índice
index = pinecone.GRPCIndex(index_name)

# Carrega modelo de embedding
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Insere chunks no Pinecone
insert_pinecone(index, chunks, embedding_model)

# Construção de *chains*

### Chain1: k = 3

In [ ]:
# Carrega modelo de embedding
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Conecta ao Pinecone
index_name = "noticias-icmc"
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

# Carrega o index do Pinecone (vector database)
vectorstore = Pinecone.from_existing_index(index_name, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Carrega LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Define template de prompt
template = """Use os seguintes trechos de contexto para responder à pergunta no final.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Contexto: {contexto}
Pergunta: {pergunta}
"""

prompt_template =  ChatPromptTemplate.from_template(template)

# Define chain
chain1 = {
    "contexto": itemgetter("pergunta") | retriever,
    "pergunta": itemgetter("pergunta")
} | prompt_template | llm | StrOutputParser()

### Chain 2: k = 7

In [ ]:
# Carrega modelo de embedding
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Conecta ao Pinecone
index_name = "noticias-icmc"
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

# Carrega o index do Pinecone (vector database)
vectorstore = Pinecone.from_existing_index(index_name, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 7})

# Carrega LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Define template de prompt
template = """Use os seguintes trechos de contexto para responder à pergunta no final.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Contexto: {contexto}
Pergunta: {pergunta}
"""

prompt_template =  ChatPromptTemplate.from_template(template)

# Define chain
chain2 = {
    "contexto": itemgetter("pergunta") | retriever,
    "pergunta": itemgetter("pergunta")
} | prompt_template | llm | StrOutputParser()

### Chain 3: k = 4

In [ ]:
# Carrega modelo de embedding
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Conecta ao Pinecone
index_name = "noticias-icmc"
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

# Carrega o index do Pinecone (vector database)
vectorstore = Pinecone.from_existing_index(index_name, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# Carrega LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Define template de prompt
template = """Use os seguintes trechos de contexto para responder à pergunta no final.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Contexto: {contexto}
Pergunta: {pergunta}
"""

prompt_template =  ChatPromptTemplate.from_template(template)

# Define chain
chain3 = {
    "contexto": itemgetter("pergunta") | retriever,
    "pergunta": itemgetter("pergunta")
} | prompt_template | llm | StrOutputParser()

# Experiências

### Notícia 1
Isolamento e coesão facilitaram propagação de informações dos grupos de direita nas redes

In [ ]:
chain1.invoke({"pergunta": "Qual a principal característica de um grupo político polarizado?"})

'A principal característica de um grupo político polarizado é o isolamento de outros grupos.'

In [ ]:
chain1.invoke({"pergunta": "O que fez com que os grupos de esquerda tivessem menos sucesso do que os grupos de direitas nas redes sociais durantes as eleições brasileiras de 2022? A resposta deve ser curta. No final da resposta referencie a fonte das informações e seu link."})

'Os grupos de esquerda tiveram menos sucesso nas redes sociais durante as eleições brasileiras de 2022 devido à sua estrutura hierárquica menos clara e à sua integração com outras comunidades online. Por outro lado, os grupos de direita eram mais isolados e coesos internamente, o que permitiu uma propagação coordenada de informações mais rápida e eficiente. Isso é evidenciado pela análise matemática das redes feita pelos pesquisadores do Instituto de Ciências Matemáticas e de Computação (ICMC) da USP. Fonte: Jornal da USP - https://jornal.usp.br/ciencias/ciencias-isolamento-e-coesao-dos-grupos-de-direita-facilitaram-propagacao-coordenada-nas-eleicoes/'

In [ ]:
chain1.invoke({"pergunta": "O que fez com que os grupos de esquerda tivessem menos sucesso do que os grupos de direitas nas redes sociais durantes as eleições brasileiras de 2022? No final da resposta cite a fonte das informações."})

'De acordo com a pesquisa realizada no Instituto de Ciências Matemáticas e de Computação (ICMC) da USP, os grupos de esquerda apresentaram menos sucesso nas redes sociais durante as eleições brasileiras de 2022 devido a dois fatores principais. Primeiro, os grupos de esquerda eram maiores em tamanho e tinham uma estrutura hierárquica menos clara, o que os tornava mais descentralizados. Em contraste, os grupos de direita eram mais isolados e coesos internamente, com uma hierarquia mais rígida e controlados por um número menor de influenciadores. Essa estrutura hierárquica mais clara nos grupos de direita permitia uma propagação coordenada de informações mais rápida e eficiente. Além disso, os grupos de esquerda estavam mais misturados com outras comunidades on-line, enquanto os grupos de direita eram mais isolados. Essas conclusões foram detalhadas em um artigo na revista científica Journal of Physics: Complexity, publicado em 13 de setembro de 2023. A fonte das informações é o Jornal d

### Notícia 2

Atenção à Saúde Mental e Inclusão na Universidade

In [ ]:
chain1.invoke({"pergunta": "Ocorreu algum evento entre os dias 3 e 4 de outubro de 2023?"})

'Não é possível determinar se ocorreu algum evento entre os dias 3 e 4 de outubro de 2023 com base nos trechos de contexto fornecidos.'

In [ ]:
chain2.invoke({"pergunta": "Ocorreu algum evento entre os dias 3 e 4 de outubro de 2023?"})

'Sim, ocorreu um evento intitulado "Atenção à Saúde Mental e Inclusão na Universidade" nos dias 3 e 4 de outubro de 2023.'

In [ ]:
chain3.invoke({"pergunta": "Ocorreu algum evento entre os dias 3 e 4 de outubro?"})

'Sim, ocorreu um evento intitulado "Atenção à Saúde Mental e Inclusão na Universidade" nos dias 3 e 4 de outubro.'

In [ ]:
chain1.invoke({"pergunta": "Quando ocorreu o evento 'Atenção à Saúde Mental e Inclusão na Universidade'?"})

"O evento 'Atenção à Saúde Mental e Inclusão na Universidade' ocorreu nos dias 3 e 4 de outubro de 2023."

In [ ]:
get_sources(vectorstore, "Ocorreu algum evento entre os dias 3 e 4 de outubro?", 4)

[Document(page_content='Em entrevista ao\xa0Jornal da USP, o pesquisador Ruben Interian, do ICMC, autor do trabalho, acredita que as conclusões do estudo podem ser aplicadas para entender os acontecimentos em Brasília em 8 de janeiro deste ano. O julgamento dos acusados de depredar as sedes dos Três Poderes, como parte de uma tentativa de golpe de Estado, começou no último dia 13 de setembro, no Supremo Tribunal Federal (STF). “É necessário esclarecer que, em um estudo dessa natureza, precisamos ter muito cuidado em apontar uma relação causal entre determinadas características da rede de interação em ambiente on-line e ações dessas pessoas no mundo real”, afirma. “Porém, parece claro que uma estrutura hierárquica e centralizada de comunicação facilita, ao menos em parte, ações radicalizadas de grandes grupos de pessoas com alto grau de coesão interna.” \n\n\n\n\n\n\n\n\n Ruben Interian - Foto: Currículo Lattes', metadata={'language': 'pt-BR', 'source': 'https://jornal.usp.br/ciencias/c

### Notícia 3

Projeto Temático une Ciência de Dados e Sociologia no mapeamento da criminalidade

In [ ]:
chain1.invoke({"pergunta": "O que significam as siglas NEV e CeMEAI?"})

'NEV significa Núcleo de Estudos da Violência da USP, e CeMEAI significa Centro de Ciências Matemáticas Aplicadas à Indústria.'

In [ ]:
chain1.invoke({"pergunta": "Qual o nome, as frentes e quanto tempo durará o projeto desenvolvido em conjunto pelo NEV e CeMEAI?"})

'O nome do projeto desenvolvido em conjunto pelo NEV e CeMEAI é "Criminalidade, Insegurança e Legitimidade: uma abordagem transdisciplinar". As frentes de pesquisa do projeto são a legitimidade e a impunidade, os padrões urbanos e criminais, além de incluir um portal de dados para organizar e analisar informações relacionadas à criminalidade, e o treinamento multidisciplinar de estudantes e pesquisadores. O projeto terá duração de cinco anos.'

### Notícia 4

O avanço das pesquisas matemáticas com foco no espectro autista

In [ ]:
chain1.invoke({"pergunta": "A pesquisa de Francisco Rodrigues utilizou a imagem cerebral de quantas pessoas?"})

'A pesquisa de Francisco Rodrigues utilizou a imagem cerebral de 500 pessoas.'

In [ ]:
chain1.invoke({"pergunta": "Quantas das imagens cerebrais utilizadas pela pesquisa de Francisco Rodrigues eram de pessoas no espectro autista?"})

'A pesquisa de Francisco Rodrigues utilizou dados de imagens cerebrais de 242 pessoas pertencentes ao espectro autista.'

In [ ]:
chain1.invoke({"pergunta": "Quantos porcento da imagens cerebrais utilizadas na pesquisa feita por Francisco Rodrigues eram referentes a pessoas no espectro autista?"})

'A pesquisa feita por Francisco Rodrigues utilizou dados de imagens cerebrais de 500 pessoas, sendo 242 pertencentes ao espectro autista. Portanto, 48,4% das imagens cerebrais utilizadas na pesquisa eram referentes a pessoas no espectro autista.'

In [ ]:
chain1.invoke({"pergunta": "Obtenha a porcentagem das imagens cerebrais utilizadas na pesquisa feita por Francisco Rodrigues que eram referentes a pessoas no espectro autista e tire a raiz quadrada desse valor em porcentagem."})

'A porcentagem das imagens cerebrais utilizadas na pesquisa feita por Francisco Rodrigues que eram referentes a pessoas no espectro autista é de 48,4%. Portanto, a raiz quadrada desse valor em porcentagem é aproximadamente 6,96%.'

### Perguntas que não podem ser respondidas

In [ ]:
chain1.invoke({"pergunta": "Onde Bruno nasceu?"})

'Não há informações suficientes no contexto fornecido para determinar onde Bruno nasceu.'

In [ ]:
chain1.invoke({"pergunta": "Qual a capital do Brasil?"})

'Não há informações suficientes nos trechos de contexto fornecidos para responder à pergunta sobre qual é a capital do Brasil.'

In [ ]:
chain1.invoke({"pergunta": "Quanto é 2 + 2?"})

'Desculpe, mas não tenho a resposta para essa pergunta.'